In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


import tensorflow as tf
from tensorflow.keras.datasets.fashion_mnist import load_data

In [ ]:
mnistData = load_data()
(x_train,y_train),(x_test,y_test) = mnistData

In [ ]:
x_trainR=x_train.reshape(-1,28*28)
x_testR=x_test.reshape(-1,28*28)
y_trainOHE = tf.keras.utils.to_categorical(y_train)
y_testOHE = tf.keras.utils.to_categorical(y_test)

In [ ]:
# 파이토치 하이(클래스기반)
# => f1_score(train,test)
# => confusion matrix(trian,test)

import torch
from torch.optim import Adam
from torch.nn import Linear, Sequential, Module, CrossEntropyLoss, Softmax, ReLU

class MultiClass(Module):
    """
    mutli_classification
    fashion_mnist => flatten (28*28)
    (28*28) => (128)
    (128) => (64)
    (64) => (10)
    """
    def __init__(self) -> None:
        super().__init__()    
        self.nn1 = Linear(28*28,128)
        self.nn2 = Linear(128,64)
        self.nn3 = Linear(64,10)
        self.relu1 = ReLU()
        self.relu2 = ReLU()
        self.sm1 = Softmax(dim=1)
    def forward(self, x):
        x = self.nn1(x)
        x = self.relu1(x)
        
        x = self.nn2(x)
        x = self.relu1(x)
        
        x = self.nn3(x)
        x = self.sm1(x)
        return x
    
x_train_tensor = torch.FloatTensor(x_trainR)
x_test_tensor = torch.FloatTensor(x_testR)

y_train_tensor = torch.FloatTensor(y_trainOHE)
y_test_tensor = torch.FloatTensor(y_testOHE)


model = MultiClass()
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 1e-3)

for epoch in range(2000):
    optimizer.zero_grad()
    x_hat = model(x_train_tensor)
    cost = loss_fn(x_hat,y_train_tensor)
    cost.backward()
    optimizer.step()
    acc = accuracy_score(
        x_hat.argmax(axis=1).detach().numpy(),
        y_train_tensor.argmax(axis=1).detach().numpy()
    )
    print(f'epoch => {epoch} / cost => {cost.item()} / acc => {acc}')

In [ ]:
print('train_data')
f1_train = round(f1_score(x_hat.argmax(axis=1).detach().numpy(),y_train,average='macro'),5)
print(f'f1_score : {f1_train}')
acc_train = round(accuracy_score(x_hat.argmax(axis=1).detach().numpy(),y_train),5)
print(f'accuracy : {acc_train}')
sns.heatmap(
    confusion_matrix(x_hat.argmax(axis=1).detach().numpy(),y_train),
    annot=True,
    fmt='d'
)
plt.show()
print('-'*30)
x_hat = model(x_test_tensor)
print('test_data')
f1_test = round(f1_score(x_hat.argmax(axis=1).detach().numpy(),y_test,average='macro'),5)
print(f'f1_score : {f1_test}')
acc_test = round(accuracy_score(x_hat.argmax(axis=1).detach().numpy(),y_test),5)
print(f'accuracy : {acc_test}')
sns.heatmap(
    confusion_matrix(x_hat.argmax(axis=1).detach().numpy(),y_test),
    annot=True,
    fmt='d'
)
plt.show()
print('-'*30)